## Sliding 1D

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pylops

from pympler import asizeof
from pylops.signalprocessing.sliding1d import sliding1d_design
from pylops.utils.describe import describe

from pylops.signalprocessing import Sliding1D
from sliding1dold import Sliding1D as Sliding1DOLD

USE_CUPY = False

In [2]:
if USE_CUPY:
    import cupy as np
    from cupyx.profiler import benchmark
    np_asarray = np.asarray
    np_asnumpy = np.asnumpy
    np_float = np.float32
    np_floatc = np.complex64
    mempool = np.get_default_memory_pool()
else:
    np_asarray = np.asarray
    np_asnumpy = np.asarray
    np_float = np.float64
    np_floatc = np.complex128

In [3]:
def bench_Op(Op, x):
    return Op @ x

def bench_OpH(Op, x):
    return Op.H @ x

In [4]:
savetaper = True
tapertype = 'cosine' #None#

In [5]:
#nwins = 50 # small
nwins = 500 # large
nwin = 26
nover = 3
nop = 64
dimd = nwin * nwins - 3 * nover

t = np.arange(dimd) * 0.004
y = np.sin(2 * np.pi * 20 * t)

nwins, dim, mwin_inends, dwin_inends = sliding1d_design(dimd, nwin, nover, (nop + 2) // 2)

# no operator broadcast
Op = pylops.signalprocessing.FFT(nwin, nfft=nop, real=True)
Slid = Sliding1DOLD(Op.H, dim, dimd, nwin, nover, tapertype=tapertype)
Slid1a = Sliding1D(Op.H, dim, dimd, nwin, nover, tapertype=tapertype, savetaper=savetaper)

# with operator broadcast
Op = pylops.signalprocessing.FFT((nwins, nwin), nfft=nop, axis=-1, real=True)
Slid1b = Sliding1D(Op.H, dim, dimd, nwin, nover, tapertype=tapertype, savetaper=savetaper)

x = Slid.H * y.ravel()

   276   299   322   345   368   391   414   437   460   483   506   529
   552   575   598   621   644   667   690   713   736   759   782   805
   828   851   874   897   920   943   966   989  1012  1035  1058  1081
  1104  1127  1150  1173  1196  1219  1242  1265  1288  1311  1334  1357
  1380  1403  1426  1449  1472  1495  1518  1541  1564  1587  1610  1633
  1656  1679  1702  1725  1748  1771  1794  1817  1840  1863  1886  1909
  1932  1955  1978  2001  2024  2047  2070  2093  2116  2139  2162  2185
  2208  2231  2254  2277  2300  2323  2346  2369  2392  2415  2438  2461
  2484  2507  2530  2553  2576  2599  2622  2645  2668  2691  2714  2737
  2760  2783  2806  2829  2852  2875  2898  2921  2944  2967  2990  3013
  3036  3059  3082  3105  3128  3151  3174  3197  3220  3243  3266  3289
  3312  3335  3358  3381  3404  3427  3450  3473  3496  3519  3542  3565
  3588  3611  3634  3657  3680  3703  3726  3749  3772  3795  3818  3841
  3864  3887  3910  3933  3956  3979  4002  4025  4

In [6]:
print(np.allclose(Slid @ x, Slid1a @ x), np.allclose(Slid.H @ y, Slid1a.H @ y))
print(np.allclose(Slid @ x, Slid1b @ x), np.allclose(Slid.H @ y, Slid1b.H @ y))

True True
True True


In [7]:
if not USE_CUPY:
    %timeit -n 5 -r 50 Slid * x # OLD
    %timeit -n 5 -r 50 Slid1a * x # NEW
    %timeit -n 5 -r 50 Slid1b * x # NEW with Op broadcasted
else:
    print(benchmark(bench_Op, (Slid, x,), n_repeat=500))
    print(benchmark(bench_Op, (Slid1a, x,), n_repeat=500))
    print(benchmark(bench_Op, (Slid1b, x,), n_repeat=500))

24.6 ms ± 236 µs per loop (mean ± std. dev. of 50 runs, 5 loops each)
12.3 ms ± 175 µs per loop (mean ± std. dev. of 50 runs, 5 loops each)
1.17 ms ± 49.9 µs per loop (mean ± std. dev. of 50 runs, 5 loops each)


In [8]:
if not USE_CUPY:
    %timeit -n 5 -r 50 Slid.H * y # OLD
    %timeit -n 5 -r 50 Slid1a.H * y # NEW
    %timeit -n 5 -r 50 Slid1b.H * y # NEW with Op broadcasted
else:
    print(benchmark(bench_OpH, (Slid, y,), n_repeat=500))
    print(benchmark(bench_OpH, (Slid1a, y,), n_repeat=500))
    print(benchmark(bench_OpH, (Slid1b, y,), n_repeat=500))

13.1 ms ± 165 µs per loop (mean ± std. dev. of 50 runs, 5 loops each)
7.47 ms ± 257 µs per loop (mean ± std. dev. of 50 runs, 5 loops each)
344 µs ± 17.2 µs per loop (mean ± std. dev. of 50 runs, 5 loops each)


In [9]:
print(asizeof.asizeof(Slid) * 1e-6, asizeof.asizeof(Slid1a) * 1e-6, asizeof.asizeof(Slid1b) * 1e-6) 

1.2774079999999999 0.132264 0.132376
